In [1]:
import numpy as np
import  pandas as pd
import tensorflow as tf,keras
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from plot_keras_history import plot_history
%matplotlib notebook
import matplotlib.pyplot as plt
from keras.layers import Bidirectional

In [2]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [2]:
data=pd.read_csv('DATA.csv')

In [3]:
data.head(5)

,Unnamed: 0,t_dist,combined_result,label
0,0,0.00,0.013,1.0
1,1,0.01,0.000,1.0
2,2,0.02,0.004,1.0
3,3,0.03,0.005,1.0
4,4,0.04,0.000,1.0


In [4]:
data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [5]:
data.head(5)

,t_dist,combined_result,label
0,0.00,0.013,1.0
1,0.01,0.000,1.0
2,0.02,0.004,1.0
3,0.03,0.005,1.0
4,0.04,0.000,1.0


In [6]:
data['label'].value_counts()

label
0.0    250225
1.0     29862
2.0     19908
Name: count, dtype: int64

In [7]:
data['label']=data['label'].replace(2,1)

In [8]:
data['label'].value_counts()

label
0.0    250225
1.0     49770
Name: count, dtype: int64

In [9]:
X=data['combined_result']
y=data['label']
time_series=data['t_dist']

In [11]:
y.shape

(299995,)

In [10]:
plt.figure(figsize=(10,6))
plt.plot(X,label='Actual Signal',color='blue')
plt.plot(y,label='Predicted Labels',color='red',linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [12]:
y.value_counts()

label
0.0    250225
1.0     49770
Name: count, dtype: int64

In [13]:
y[0]

1.0

In [59]:
# Logic for creating Sequences
# Here i have got time for both label 1 and 0
sequences=[]
labels=[]
temp=[]
for index,value in y.items():
    if value==1:
        temp.append(X[index])
        if (index+1)<len(y) and y[index+1]==0:
            sequences.append(temp)
            labels.append(1)
            temp=[]
    if value==0:
        temp.append(X[index])
        if (index+1)<len(y) and y[index+1]==1:
            sequences.append(temp)
            labels.append(0)
            temp=[]

In [63]:
# Not a sequence but label is 1 
zero_seq_label_one=[]
for position, sublist in enumerate(sequences):
    if all(element == 0 for element in sublist):
        if labels[position]==1:
            zero_seq_label_one.append(position)

In [64]:
len(zero_seq_label_one)

0

In [62]:
# Removing those
sequences = [row for i, row in enumerate(sequences) if i not in zero_seq_label_one]
labels = [val for i, val in enumerate(labels) if i not in zero_seq_label_one]

In [68]:
# Is a sequence but label 0
seq_label_zero=[]
for position, sublist in enumerate(sequences):  
    if any(element != 0 for element in sublist):
        if labels[position]==0:
            seq_label_zero.append(position)

In [70]:
len(sequences)

1391

In [67]:
# Replacing those with 1
for value in seq_label_zero:
    labels[value]=1

In [71]:
zero=0
one=0
for value in labels:
    if value==0:
        zero+=1
    if value==1:
        one+=1
print(zero)
print(one)

597
794


In [16]:
largest_length = 0
shortest_length = float('inf')  # Set to positive infinity initially

for sublist in sequences:
    sublist_length = len(sublist)

    # Update largest_length if necessary
    if sublist_length > largest_length:
        largest_length = sublist_length

    # Update shortest_length if necessary
    if sublist_length < shortest_length:
        shortest_length = sublist_length

print("Largest sublist length:", largest_length)
print("Shortest sublist length:", shortest_length)

Largest sublist length: 751
Shortest sublist length: 16


In [56]:
# Splitting Dataset
X_train,X_test,y_train,y_test=train_test_split(sequences,labels,test_size=0.2,random_state=42)

In [57]:
# # calculating time in X_test for only events i.e where label for given sequence is 1
# test_time=[]
# index_seq=[]
# for pos, item in enumerate(sequences):
#     for lis in X_test:
#         if labels[pos]==1 and item==lis:
#             test_time.append(time[pos])
#             index_seq.append(pos)
#     
    

In [58]:
# test_time[0]

In [59]:
# time[30]

In [60]:
print(len(np.unique(X_train[78])))
print(X_train[78])

19
[0.027, 0.034, 0.001, 0.026, 0.075, 0.028, 0.005, 0.039, 0.004, 0.014, 0.009, 0.004, 0.006, 0.002, 0.003, 0.002, 0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.026, 0.038, 0.001, 0.026, 0.073, 0.03, 0.006, 0.039, 0.004, 0.014, 0.01, 0.004, 0.006, 0.002, 0.003, 0.001, 0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [61]:
# Padding If Needed
X_train=pad_sequences(X_train,padding='post',truncating='post',dtype=float,maxlen=50)
X_test=pad_sequences(X_test,padding='post',truncating='post',dtype=float,maxlen=50)

In [62]:
print(len(np.unique(X_train[78])))
X_train.shape

19


(1159, 50)

In [63]:
y_train=np.array(y_train)
y_test=np.array(y_test)
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

In [64]:
print(y_train.shape)

(1159, 1)


In [65]:
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test=X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
print(X_train)

[[[0.   ]
  [0.   ]
  [0.   ]
  ...
  [0.   ]
  [0.   ]
  [0.   ]]

 [[0.   ]
  [0.   ]
  [0.   ]
  ...
  [0.   ]
  [0.   ]
  [0.   ]]

 [[0.   ]
  [0.   ]
  [0.   ]
  ...
  [0.   ]
  [0.   ]
  [0.   ]]

 ...

 [[0.005]
  [0.005]
  [0.001]
  ...
  [0.   ]
  [0.   ]
  [0.   ]]

 [[0.007]
  [0.001]
  [0.016]
  ...
  [0.001]
  [0.   ]
  [0.001]]

 [[0.002]
  [0.004]
  [0.002]
  ...
  [0.   ]
  [0.   ]
  [0.   ]]]


In [66]:
# Attention Mechanism
from keras.layers import *
from keras.models import *
from keras import backend as K
import keras
tf.keras.saving.get_custom_objects().clear()

In [67]:
#@tf.keras.utils.register_keras_serializable(package="Attention")
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='he_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context
    

In [68]:
model=Sequential([
    Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True) ,input_shape=(X_train.shape[1],1)),# 64
    Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True)),
    attention(),
    tf.keras.layers.Dense(1,activation='sigmoid')
],name="Time-Series-Model")
model.summary()

Model: "Time-Series-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 50, 128)          33792     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 50, 64)           41216     
 nal)                                                            
                                                                 
 attention (attention)       (None, 64)                114       
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 75,187
Trainable params: 75,187
Non-trainable params: 0
_________________________________________________________________


In [69]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

In [73]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001)#0.0005,0.001
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [31]:
model_history=model.fit(X_train,y_train,epochs=100,batch_size=16,validation_data=(X_test,y_test),callbacks=[early_stopping])# batch_size=32

Epoch 1/100
73/73 [==============================] - 13s 68ms/step - loss: 0.6927 - accuracy: 0.5047 - val_loss: 0.6922 - val_accuracy: 0.4966
Epoch 2/100
73/73 [==============================] - 3s 44ms/step - loss: 0.6922 - accuracy: 0.5013 - val_loss: 0.6915 - val_accuracy: 0.4966
Epoch 3/100
73/73 [==============================] - 4s 53ms/step - loss: 0.6916 - accuracy: 0.5013 - val_loss: 0.6908 - val_accuracy: 0.4966
Epoch 4/100
73/73 [==============================] - 4s 52ms/step - loss: 0.6910 - accuracy: 0.5255 - val_loss: 0.6900 - val_accuracy: 0.4966
Epoch 5/100
73/73 [==============================] - 4s 48ms/step - loss: 0.6903 - accuracy: 0.5013 - val_loss: 0.6891 - val_accuracy: 0.4966
Epoch 6/100
73/73 [==============================] - 3s 47ms/step - loss: 0.6895 - accuracy: 0.6178 - val_loss: 0.6880 - val_accuracy: 0.8966
Epoch 7/100
73/73 [==============================] - 4s 48ms/step - loss: 0.6885 - accuracy: 0.5979 - val_loss: 0.6868 - val_accuracy: 0.8586
Epoch

In [71]:
model.save_weights('Sequence-model.h5')
#model.load_weights('Sequence-model.h5')

OSError: Unable to open file (file signature not found)

In [32]:
plot_history(model_history,show_standard_deviation=False,show_average=True)

NameError: name 'model_history' is not defined

In [33]:
y_pred=model.predict(X_test)
y_pred=np.round(y_pred)
report = classification_report(y_pred,y_test)
print(report)

10/10 [==============================] - 2s 13ms/step
              precision    recall  f1-score   support

         0.0       1.00      0.83      0.91       175
         1.0       0.80      1.00      0.89       115

    accuracy                           0.90       290
   macro avg       0.90      0.92      0.90       290
weighted avg       0.92      0.90      0.90       290


In [34]:
pred_count=np.count_nonzero(y_pred==1)
test_count=np.count_nonzero(y_test==1)

In [35]:
print(pred_count/test_count) 

0.7986111111111112


In [ ]:
# 966 is vocab size, 32 is size defined for output vector, 751 is length of sequence
vocab_size=966
embedding_dim=2
max_length=50
tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim)

Working On 10 min Data

In [36]:
min_10_data=pd.read_csv('10_minutes_final_processed_gsmfloor_data5.csv')

In [37]:
min_10_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
min_10_data.head(5)

,t_dist,combined_result,label
0,0.00,0.015,1.0
1,0.01,0.016,1.0
2,0.02,0.000,1.0
3,0.03,0.021,1.0
4,0.04,0.070,1.0


In [38]:
min_10_data['label']=min_10_data['label'].replace(2,1)

In [39]:
x_values=min_10_data['combined_result']
y_values=min_10_data['label']

In [40]:
x_values[0]

0.015

In [41]:
events=[]
labe=[]
temp=[]
for index,value in y_values.items():
    if value==1:
        temp.append(x_values[index])
        if (index+1)<len(y_values) and y_values[index+1]==0:
            events.append(temp)
            labe.append(1)
            temp=[]
    if value==0:
        temp.append(x_values[index])
        if (index+1)<len(y_values) and y_values[index+1]==1:
            events.append(temp)
            labe.append(0)
            temp=[]

In [42]:
events[0]

[0.015,
 0.016,
 0.0,
 0.021,
 0.07,
 0.115,
 0.043,
 0.007,
 0.004,
 0.0,
 0.001,
 0.0,
 0.065,
 0.006,
 0.003,
 0.011,
 0.062,
 0.069,
 0.029,
 0.002,
 0.005,
 0.0,
 0.001,
 0.0]

In [43]:
events=pad_sequences(events,padding='post',dtype=float,maxlen=50)

In [44]:
labe=np.array(labe)
labe=labe.reshape(-1,1)

In [45]:
labe[0]

array([1])

In [46]:
events=events.reshape(events.shape[0], events.shape[1], 1)

In [47]:
prediction_10_min=model.predict(events)

10/10 [==============================] - 0s 15ms/step


In [48]:
prediction_10_min=np.round(prediction_10_min)

In [49]:
prediction_10_min_count=np.count_nonzero(prediction_10_min==1)
orignal_10_min_count=np.count_nonzero(labe==1)

In [50]:
print(prediction_10_min_count,"\n",orignal_10_min_count)

111 
 145


Working with 96 hours data

In [34]:
from sklearn.preprocessing import MinMaxScaler

In [57]:
real_data=pd.read_csv('new_96hrs_new_final_processed_data.csv')
real_data.head(5)

,Unnamed: 0,time,gsm,S1,S2,S3,dS1,dS2,dS3
0,0,2022-01-21 09:30:00.000000,5.075708e-07,-0.106049,-0.978210,0.182831,-0.000519,0.000000,-0.000488
1,1,2022-01-21 09:30:00.000000,1.252629e-06,-0.106567,-0.978210,0.182343,0.000000,0.000488,0.001007
2,2,2022-01-21 09:30:00.000000,1.222827e-06,-0.106567,-0.977722,0.183350,0.000519,0.000000,0.000977
3,3,2022-01-21 09:30:00.000000,3.488734e-06,-0.106049,-0.977722,0.184326,-0.001007,-0.000488,-0.001495
4,4,2022-01-21 09:30:00.000000,2.474524e-06,-0.107056,-0.978210,0.182831,-0.001495,0.000488,0.000000


In [58]:
scaler = MinMaxScaler(feature_range=(0.0, 10.789))
real_data['gsm']=scaler.fit_transform(real_data[['gsm']])
graph_data=real_data['gsm']
#graph_data=graph_data[0:2500]

In [59]:
graph_data.head(5)

0    0.002244
1    0.005539
2    0.005407
3    0.015427
4    0.010942
Name: gsm, dtype: float64

In [51]:
df=pd.DataFrame({'DATA.csv':X,"Real-world": graph_data})


In [41]:
df.plot()

# Customize the plot (add labels, titles, legends, etc. as needed)
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')
plt.title('Combined Series Plot')
plt.legend()

# Show the plot
plt.show()

<IPython.core.display.Javascript object>

In [38]:
real_sequence=[]
temp=[]
for index,value in graph_data.items():
    if value!=0.0:
        temp.append(value)
        if (index+1)<len(graph_data) and graph_data[index+1]==0.0:
            real_sequence.append(temp)
            temp=[]
    if value==0.0:
        temp.append(value)
        if (index+1)<len(graph_data) and graph_data[index+1]!=0.0:
            real_sequence.append(temp)
            temp=[]
        
    

In [39]:
real_sequence=pad_sequences(real_sequence,padding='post',dtype=float,maxlen=50)

In [40]:
# real_sequence=real_sequence[0:2000,:]

In [67]:
print(real_sequence[16110])

[[0.00105427]
 [0.00105427]
 [0.02038117]
 [0.01875446]
 [0.00329871]
 [0.00079894]
 [0.00421708]
 [0.00224856]
 [0.0065933 ]
 [0.00329871]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]


In [42]:
real_sequence=real_sequence.reshape(real_sequence.shape[0], real_sequence.shape[1], 1)

In [43]:
real_sequence_pred=model.predict(real_sequence)
real_sequence_pred=np.round(real_sequence_pred)

5075/5075 [==============================] - 82s 16ms/step


In [73]:
real_sequence_pred_count=np.count_nonzero(real_sequence_pred==0)
print(graph_data.shape)

(1915618,)


In [71]:
plt.figure(figsize=(10,6))
plt.plot(graph_data,label='Actual Signal',color='blue')
plt.plot(real_sequence_pred,label='Predicted Labels',color='red',linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(real_sequence.flatten(),label='Signal-Values',color='blue')
for i,pred in  enumerate(real_sequence_pred):
    if pred==1:
        plt.plot(i,real_sequence[i].T,'ro')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
# Till now best is with batch 16 2 biLSTM64,32 and one attention

In [ ]:
# ==================================Prediction Starts============================

In [80]:
df=pd.read_csv('10_minutes_final_processed_gsmfloor_data5.csv')
df.head(5)

,Unnamed: 0,t_dist,combined_result,label
0,0,0.00,0.015,1.0
1,1,0.01,0.016,1.0
2,2,0.02,0.000,1.0
3,3,0.03,0.021,1.0
4,4,0.04,0.070,1.0


In [81]:
df.drop(columns=['Unnamed: 0','t_dist'],axis=1,inplace=True)

In [82]:
data['label'].value_counts()

0.0    250225
1.0     49770
Name: label, dtype: int64

In [83]:
aex=data['combined_result']
yew=data['label']

In [84]:
aex.nunique()

965

In [85]:
# Logic for creating Sequences
test_seq=[]
test_label=[]
temp=[]
for index,value in yew.items():
    if value==1:
        temp.append(aex[index])
        if (index+1)<len(yew) and yew[index+1]==0:
            test_seq.append(temp)
            test_label.append(1)
            temp=[]
    if value==0:
        temp.append(aex[index])
        if (index+1)<len(yew) and yew[index+1]==1:
            test_seq.append(temp)
            test_label.append(0)
            temp=[]

In [86]:
print(len(test_seq[1]))
print(len(yew[25:390]))
# After checking this I think all is well
print(len(test_seq),"\n",len(test_label))

365
365
1449 
 1449


In [87]:
test_seq[0]

[0.013,
 0.0,
 0.004,
 0.005,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.013,
 0.0,
 0.004,
 0.005,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [88]:
test_seq=pad_sequences(test_seq,padding='post',dtype=float,maxlen=50)

In [89]:
test_seq=test_seq.reshape(test_seq.shape[0], test_seq.shape[1], 1)

In [94]:
test_seq.shape

(1449, 50, 1)

In [56]:
pred=model.predict(test_seq)
pred=np.round(pred)

46/46 [==============================] - 1s 17ms/step


In [57]:
print(pred)

[[1.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [1.]]


In [74]:
pred[8]

array([1.], dtype=float32)

In [75]:
test_label[8]

1

In [90]:
test_label=np.array(test_label)
test_label=test_label.reshape(-1,1)

In [78]:
model.evaluate(test_seq,test_label)

46/46 [==============================] - 1s 17ms/step - loss: 0.2916 - accuracy: 0.8889


[0.291627436876297, 0.8888888955116272]

In [1]:
pred=model.predict(test_seq)
pred=np.round(pred)
report = classification_report(test_label,pred)
print(report)

No GPU available.
